In [2]:
#Import libraries
import pandas as pd
import boto3
import sagemaker
from IPython.display import display, HTML

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
!pip install pandasql

In [4]:
from pandasql import sqldf

In [5]:
#Read data file and drop album name column due to characters 
df = pd.read_csv('aai-540-homework/homework-2-1/data/dataset.csv')
df=df.drop(columns='album_name')
df.head(5)

,Unnamed: 0,track_id,artists,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [6]:
#View Nan values 
df.isna().sum()

Unnamed: 0          0
track_id            0
artists             1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

In [7]:
#Drop rows with Nan values as there are only 2
df = df.dropna() 
df.isna().sum()

Unnamed: 0          0
track_id            0
artists             0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

In [8]:
#View data types for features 
df.dtypes

Unnamed: 0            int64
track_id             object
artists              object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object

In [9]:
#Python data-frame format queries
#1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99
df[['artists','track_name','popularity']].query('popularity >=99')

,artists,track_name,popularity
20001,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100
51664,Bizarrap;Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52",99
81051,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100


In [10]:
#2. List artists with an average popularity of 92
df[['artists','popularity']].groupby(['artists']).mean().query('popularity == 92')

,popularity
artists,
Harry Styles,92.0
Rema;Selena Gomez,92.0


In [11]:
#3. List the Top 10 most energetic genres
df[['energy','track_genre']].groupby(['track_genre']).mean().sort_values('energy', ascending=False).head(10)

,energy
track_genre,
death-metal,0.931470
grindcore,0.924201
metalcore,0.914485
happy,0.910971
hardstyle,0.901246
drum-and-bass,0.876635
black-metal,0.874897
heavy-metal,0.874003
party,0.871237


In [12]:
#4. How many tracks is Bad Bunny on?
df['artists'].value_counts()['Bad Bunny']

48

In [13]:
#5. Show the top 10 genres in terms of popularity sorted by their most popular track
df[['popularity','track_genre','track_name']].groupby(['track_name','track_genre']).mean().sort_values('popularity', ascending=False).head(10)

popularity
track_name                            track_genre            
Unholy (feat. Kim Petras)             dance             100.0
                                      pop               100.0
Quevedo: Bzrp Music Sessions, Vol. 52 hip-hop            99.0
La Bachata                            latin              98.0
                                      reggae             98.0
                                      reggaeton          98.0
                                      latino             98.0
Tití Me Preguntó                      latino             97.0
                                      reggaeton          97.0
Me Porto Bonito                       reggaeton          97.0

In [14]:
#SQL queries
#1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99
sqldf('''SELECT artists, track_name, popularity FROM df WHERE popularity >=99''')

,artists,track_name,popularity
0,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100
1,Bizarrap;Quevedo,"Quevedo: Bzrp Music Sessions, Vol. 52",99
2,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),100


In [15]:
#2. List artists with an average popularity of 92
sqldf('''SELECT artists, AVG(popularity) FROM df WHERE popularity == 92 GROUP BY artists''')

,artists,AVG(popularity)
0,Arctic Monkeys,92.0
1,Charlie Puth;Jung Kook;BTS,92.0
2,Harry Styles,92.0
3,Nicki Minaj,92.0
4,Rema;Selena Gomez,92.0


In [16]:
#3. List the Top 10 most energetic genres
sqldf('''SELECT track_genre, AVG(energy) FROM df GROUP BY track_genre ORDER BY AVG(energy) DESC LIMIT 10''')

,track_genre,AVG(energy)
0,death-metal,0.931470
1,grindcore,0.924201
2,metalcore,0.914485
3,happy,0.910971
4,hardstyle,0.901246
5,drum-and-bass,0.876635
6,black-metal,0.874897
7,heavy-metal,0.874003
8,party,0.871237
9,j-idol,0.868677


In [17]:
#4. How many tracks is Bad Bunny on?
sqldf('''SELECT COUNT (*) FROM df WHERE artists='Bad Bunny' ''')

,COUNT (*)
0,48


In [18]:
#5. Show the top 10 genres in terms of popularity sorted by their most popular track
sqldf('''SELECT track_genre, track_name, AVG(popularity) FROM df GROUP BY track_name, track_genre ORDER BY AVG(popularity) DESC LIMIT 10''')

,track_genre,track_name,AVG(popularity)
0,dance,Unholy (feat. Kim Petras),100.0
1,pop,Unholy (feat. Kim Petras),100.0
2,hip-hop,"Quevedo: Bzrp Music Sessions, Vol. 52",99.0
3,latin,La Bachata,98.0
4,latino,La Bachata,98.0
5,reggae,La Bachata,98.0
6,reggaeton,La Bachata,98.0
7,latin,Me Porto Bonito,97.0
8,latino,Me Porto Bonito,97.0
9,reggae,Me Porto Bonito,97.0
